In [40]:
# Imports
import numpy as np
from sympy import symbols, Poly
import random

# Main functions

### Function to generate random polynomials
Generates a polynomial of up to a given degree with a specific number of terms. The coefficients of the polynomial are within the specified bounds.

In [65]:
def generate_polynomial(max_degree, num_terms, coeff_bounds):
    """
    Inputs:
    max_degree (int): The maximum degree of the polynomial.
    num_terms (int)
    coeff_bounds (tuple): (lower bound, upper bound)

    Outputs:
    coefficients (list)
    """
    # Initialize array of coeffs
    coefficients = [0] * (max_degree + 1)
    # Check number of terms
    if num_terms > max_degree + 1:
        raise ValueError("Number of terms cannot be more than the degree of the polynomial + 1")
    # Randomly choose positions for the (#) non-zero coefficients
    non_zero_positions = random.sample(range(max_degree), num_terms - 1)
    # Assign random values within the bounds to the chosen positions
    for pos in non_zero_positions:
        # Loop to make sure it is nonzero bc selecting from [-, +] includes 0
        while coefficients[pos] is None or coefficients[pos] == 0:
          coefficients[pos] = random.randint(coeff_bounds[0], coeff_bounds[1])
    # Ensure the highest degree term is non-zero
    if coefficients[-1] == 0:
        # While loop until it is nonzero
        while coefficients[-1] is None or coefficients[-1] == 0:
          coefficients[-1] = random.randint(coeff_bounds[0], coeff_bounds[1])

    return coefficients

In [69]:
# Test function
generate_polynomial(max_degree=10, num_terms=3, coeff_bounds=[-10,10])

[-7, 6, 0, 0, 0, 0, 0, 0, 0, 0, -10]

### Nondimensionalization function
Takes in array of coefficients and returns in a nondimensionalized form (only works for 3-term polynomials due to how it reduces the 2nd and third terms to have coefficients of 1).



In [72]:
def nondimensionalize_polynomial(coefficients):
    """
    Inputs:
    coefficients(list): List of coefficients of a polynomial
    num_terms (int)
    coeff_bounds (tuple): (lower bound, upper bound)

    Outputs:
    coefficients (list)
    """
    # Identify nonzero coefficients and their powers
    nonzero_coeffs = [(coeff, len(coefficients) - idx - 1) for idx, coeff in enumerate(coefficients) if coeff != 0]

    # Ensure there are exactly three nonzero coefficients
    if len(nonzero_coeffs) != 3:
        return "Polynomial must have exactly three nonzero coefficients.", None

    # Find largest, second largest powers
    sorted_coeffs = sorted(nonzero_coeffs, key=lambda x: x[1], reverse=True)
    max_power = sorted_coeffs[0][1]
    second_max_power = sorted_coeffs[1][1]

    # Compute ε
    epsilon = sorted_coeffs[0][0] / sorted_coeffs[1][0]

    # Construct the nondimensionalized polynomial
    signs = [1 if coeff[0] > 0 else -1 for coeff in sorted_coeffs]

    # Printing polynomial (commenting out because not needed)
    # nondimensionalized_poly = f"{epsilon:.2f}x^{max_power} "
    # nondimensionalized_poly += f"{'+' if signs[1] == 1 else '-'} x^{second_max_power} "
    # nondimensionalized_poly += f"{'+' if signs[2] == 1 else '-'} 1"

    # Construct new coeffs array
    new_coefficients = [0] * (max_power + 1)
    new_coefficients[max_power] = epsilon
    new_coefficients[second_max_power] = signs[1]
    new_coefficients[0] = signs[2]
    new_coefficients = list(reversed(new_coefficients))

    # Return
    # return new_coefficients, nondimensionalized_poly
    return new_coefficients

In [71]:
# Example usage
polynomial = [0, 0, -8, 0, 5, 0, 0, 0, 0, 0, 7]
nondimensionalize_polynomial(polynomial)

[-1.6, 0, 1, 0, 0, 0, 0, 0, 1]

In [75]:
print(np.roots([-7, 6, 0, 0, 0, 0, 0, 0, 0, 0, -10]))
print(np.roots([-1.6, 0, 1, 0, 0, 0, 0, 0, 1]))

[ 1.11737589+0.29180969j  1.11737589-0.29180969j  0.70135589+0.80048499j
  0.70135589-0.80048499j  0.07413512+1.00975443j  0.07413512-1.00975443j
 -0.92181274+0.31530581j -0.92181274-0.31530581j -0.54248273+0.82319706j
 -0.54248273-0.82319706j]
[-1.04767631+0.j         -0.71023674+0.59814896j -0.71023674-0.59814896j
  1.04767631+0.j          0.71023674+0.59814896j  0.71023674-0.59814896j
  0.        +0.8751764j   0.        -0.8751764j ]


In [35]:
def nondimensionalize(arr):
    # Identify the nonzero coefficients and their indices (which represent the powers of x)
    nonzero_indices = [i for i, e in enumerate(arr) if e != 0]
    nonzero_coeffs = [arr[i] for i in nonzero_indices]

    # Ensure there are exactly 3 nonzero coefficients
    if len(nonzero_coeffs) != 3:
        return "The array must contain exactly 3 nonzero coefficients.", None

    # Sort the coefficients and indices in descending order of power
    sorted_pairs = sorted(zip(nonzero_indices, nonzero_coeffs), reverse=True)
    sorted_indices, sorted_coeffs = zip(*sorted_pairs)

    # The highest, middle, and lowest power terms
    highest_power, middle_power, lowest_power = sorted_indices

    # Scale the variable x so that the middle power term becomes x (i.e., x = a^(1/(middle_power - lowest_power)) * y)
    # and calculate epsilon
    scale_factor = (sorted_coeffs[1] ** (1 / (middle_power - lowest_power)))
    epsilon = sorted_coeffs[0] * scale_factor ** (highest_power - middle_power)

    # Construct the nondimensionalized polynomial string
    nondim_poly = f"{epsilon:.1f}x^{highest_power - middle_power} + x + 1"

    return nondim_poly

# Example usage
print(nondimensionalize([0, 0, 0, 0, 0, 0, 10, 0, 0, -9, -1]))


-1.0-1.8jx^1 + x + 1


In [34]:
nondimensionalize([0, 0, 0, 0, 0, 0, 10, 0, 0, -9, -1])

'90.0x^10 + x^9 + 1'

In [31]:
# Generates sympy polynomial from
def polynomial_from_coefficients(coefficients):
    """
    Converts a list of coefficients into a polynomial expression using sympy.

    Parameters:
    coefficients (list): A list of coefficients, where the index represents the power of x.

    Returns:
    sympy expression: The polynomial expression.
    """
    x = symbols('x')
    coefficients_reverse = coefficients
    coefficients_reverse.reverse()
    polynomial = sum(coef * x**i for i, coef in enumerate(coefficients_reverse))
    return Poly(polynomial)

In [32]:
# Test functions
coefficients = generate_polynomial(degree=10, num_terms=3, coeff_bounds=[-10,10])
print("Coefficients", coefficients)

polynomial_expression = polynomial_from_coefficients(coefficients)
polynomial_expression

Coefficients [0, 0, -8, 0, 5, 0, 0, 0, 0, 0, 7]


Poly(-8*x**8 + 5*x**6 + 7, x, domain='ZZ')

## Generate latex for problem and solution

## Check that analytical solutions are approx. equal to the numerical solution.

If valid, generate latex -> add to dataset
- Check for duplicates??

Discard problem otherwise

## Unit tests for each function

### Generate polynomial
- Generate several polynomials, check that there are exactly n nonzero coefficients and that there is a constant.

### Nondimensionalize polynomial
- Generate many polynomials, nondimensionalize, compare numerically-solved roots to check they are the same (but scaled)